In [1]:
%%time
import os
if not os.path.isdir('urban_oculus'):
  !git clone https://github.com/tgandor/urban_oculus
!(cd urban_oculus; git pull)
%run urban_oculus/snippets/get_coco_val2017_nounzip.ipynb
%run urban_oculus/snippets/validate_2017_functions.py

Already up to date.
CPU times: user 2.67 ms, sys: 475 µs, total: 3.14 ms
Wall time: 126 ms
/usr/bin/mogrify
CPU times: user 1.08 s, sys: 146 ms, total: 1.22 s
Wall time: 8.25 s


In [2]:
import time
time.strftime('%Y-%m-%d %H:%M:%S')
os.getpid()


22280

In [3]:
def validate_coco2017(model_config, quality, output_dir):
  result_file = result_path(model_config, quality, output_dir)
  if os.path.exists(result_file):
    print(time.strftime('%Y-%m-%d %H:%M:%S'), 'result seems to exist:', result_file)
    print('skipping', model_config, quality)
    return

  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'unpacking')
  !unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
  # save_filesizes('filesizes_original.json', 'original', 0, output_dir)

  print(time.strftime('%Y-%m-%d %H:%M:%S'), f'degrading quality to {quality}')
  start = time.time()
  !mogrify -verbose -quality {quality} datasets/coco/val2017/*.jpg 2>&1 | tqdm --total 10000 >/dev/null
  elapsed = time.time() - start
  # save_filesizes(f'filesizes_{quality}.json', quality, elapsed, output_dir)

  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'finished', quality)
  results, inference_time =  run_validation(model_config)
  
  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'finished', quality)
  save_results(model_config, quality, results, inference_time, output_dir)

In [4]:
torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [ ]:
%%time
# https://github.com/conansherry/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml
output_dir = '.' # '/content/gdrive/My Drive/results'  # mounted or not, here I come
#model_idx = 5
#model_config = MODEL_ZOO_CONFIGS[model_idx]

for model_idx, model_config in enumerate(MODEL_ZOO_CONFIGS):
    print('model config:', model_config, 'idx:', model_idx)
    for q in tqdm(range(1, 101, 3)):
      validate_coco2017(model_config, q, output_dir)

model config: COCO-Detection/faster_rcnn_R_50_C4_3x.yaml idx: 0


2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_C4_3x_q_001.json
skipping COCO-Detection/faster_rcnn_R_50_C4_3x.yaml 1
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_C4_3x_q_004.json
skipping COCO-Detection/faster_rcnn_R_50_C4_3x.yaml 4
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_C4_3x_q_007.json
skipping COCO-Detection/faster_rcnn_R_50_C4_3x.yaml 7
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_C4_3x_q_010.json
skipping COCO-Detection/faster_rcnn_R_50_C4_3x.yaml 10
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_C4_3x_q_013.json
skipping COCO-Detection/faster_rcnn_R_50_C4_3x.yaml 13
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_C4_3x_q_016.json
skipping COCO-Detection/faster_rcnn_R_50_C4_3x.yaml 16
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_C4_3x_q_019.json
skipping COCO-Detection/faster_rcnn_R_50_C4_3x.yaml 19
2020-06-1

2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_DC5_3x_q_001.json
skipping COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml 1
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_DC5_3x_q_004.json
skipping COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml 4
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_DC5_3x_q_007.json
skipping COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml 7
2020-06-10 10:14:38 result seems to exist: ./val2017_faster_rcnn_R_50_DC5_3x_q_010.json
skipping COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml 10
2020-06-10 10:14:38 unpacking
5001it [00:03, 1351.67it/s]                                                     
2020-06-10 10:14:42 degrading quality to 13
100%|████████████████████████████████████| 10000/10000 [00:36<00:00, 274.79it/s]
2020-06-10 10:15:18 finished 13
Loading and preparing results...
DONE (t=0.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=12.87s

In [2]:
import torch

print('Device count:', torch.cuda.device_count())
torch.cuda.set_device(1)

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name())
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3, 1), 'GB')

!head -n 11 /proc/cpuinfo


Device count: 2
Using device: cuda

GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Core(TM) i9-7900X CPU @ 3.30GHz
stepping	: 4
microcode	: 0x2000065
cpu MHz		: 4013.715
cache size	: 14080 KB
physical id	: 0
siblings	: 20


In [10]:
!ls

18.py			      NewValidate_multiple_JPEG.ipynb
22.py			      nowe_id
annotations_trainval2017.zip  pycardio
AoC22.ipynb		      pytony
datasets		      Scans
environment_testing.ipynb     stare_id
euler_spoj		      swapfile
hemodynamic-pipe	      TrainSet
in18.txt		      urban_oculus
in22.txt		      val2017_faster_rcnn_R_101_FPN_3x_q_5.json
lakeit			      val2017.zip
log_18.txt		      Zarr_scratch.ipynb
meats
